In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import pandas as pd
import seaborn as sns
import keras
from keras import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score



d = pd.read_csv('houses_edited.csv')


features = ["bedrooms", "bathrooms", "sqft", "parking", 
            "mean_district_income", "Plex", "Semi-Detached", 
            "Store W/Apt/Offc", "Co-Op Apt", "Co-Ownership Apt", 
            "Link", "Comm Element Condo", "Att/Row/Twnhouse", 
            "Condo Townhouse", "Detached", "Condo Apt"] #define list of features to input in nerual network


df = pd.DataFrame(data = d)


def remove_beds(x): #function to remove the word "beds" from column of number of beds
    x = x.replace(" beds", "")
    return x
def calc_beds(j): #function to calculate number of beds from string
    answer = sum(int(i) for i in j.split('+'))
    return answer


df.bedrooms = df.bedrooms.apply(remove_beds) 
df.bedrooms = df.bedrooms.apply(calc_beds)

In [3]:
df = df.drop(["description", "mls", "bedrooms_ag", "bedrooms_bg", "full_link", "full_address", "title",
              "final_price_transformed", "final_price_log", "city_district", "district_code",
              "index", "lat", "long", "list_price"], axis = 1) #Remove insignificant values

#Fill missing sqft values based on type of property

sqft_Store = 0
sqft_Link = 0
sqft_Twn = 0
sqft_Plex = 0
sqft_Detached = 0
sqft_Semi_detached = 0
sqft_Store_count = 0
sqft_Link_count = 0
sqft_Twn_count = 0
sqft_Plex_count = 0
sqft_Detached_count = 0
sqft_Semi_detached_count = 0


for index, row in df.iterrows():
    if pd.notnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
      sqft_Store += row["sqft"]
      sqft_Store_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Link":
      sqft_Link += row["sqft"]
      sqft_Link_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
      sqft_Twn += row["sqft"]
      sqft_Twn_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Plex":
      sqft_Plex += row["sqft"]
      sqft_Plex_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Detached":
      sqft_Detached += row["sqft"]
      sqft_Detached_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Semi-Detached":
      sqft_Semi_detached += row["sqft"]
      sqft_Semi_detached_count += 1

avg_Store = sqft_Store/sqft_Store_count
avg_Link = sqft_Link/sqft_Link_count
avg_Twn = sqft_Twn/sqft_Twn_count
avg_Plex = sqft_Plex/sqft_Plex_count
avg_Detached = sqft_Detached/sqft_Detached_count
avg_Semi_detached = sqft_Semi_detached/sqft_Semi_detached_count


for index, row in df.iterrows():
  #print(row["type"], row["sqft"])
  if pd.isnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
    df.at[index, "sqft"] = avg_Store

  if pd.isnull(row["sqft"]) and row["type"] == "Link":
    df.at[index, "sqft"] = avg_Link

  if pd.isnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
    df.at[index, "sqft"] = avg_Twn

  if pd.isnull(row["sqft"]) and row["type"] == "Plex":
    df.at[index, "sqft"] = avg_Plex

  if pd.isnull(row["sqft"]) and row["type"] == "Detached":
    df.at[index, "sqft"] = avg_Detached

  if pd.isnull(row["sqft"]) and row["type"] == "Semi-Detached":
    df.at[index, "sqft"] = avg_Semi_detached


dummy_type = pd.get_dummies(df["type"]) #allow 
df = pd.concat([df, dummy_type], axis = 1)
df = df.drop(["type"], axis = 1)

In [21]:
train, test = train_test_split(df, test_size = 0.20, random_state = 0) #Split data into 2 parts, 
                                                                       #training and testing data, with ratio 8:2 

x_train = train[features]
y_train = train[['final_price']]

model = Sequential()

model.add(Dense(64, input_dim = 16,activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))

opt = keras.optimizers.Adam(learning_rate = 0.03) 

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['MeanAbsolutePercentageError'])
model.fit(x_train, y_train, epochs = 1800, batch_size = 200, shuffle = False, verbose = 2)
#training = model.predict(x_train)
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy))

predictions = model.predict(x_train)
#print(predictions)
for i in range(10):
  print("%d, expected: %d" % (predictions[i], y_train.iloc[i]))

# Predicting prices from test set
# x_test = test[features]
# y_test = test[['final_price']]


# print(model.evaluate(x_test, y_test))


#Best hyperparameter result
# MSPE: 17.95%
# Neural layers: 64, 32, 1
# Learning rate: 0.03
# Epochs: 1500
# Batch size; 200


Epoch 1/1800
61/61 - 0s - loss: 393890168832.0000 - mean_absolute_percentage_error: 44.7805
Epoch 2/1800
61/61 - 0s - loss: 238926528512.0000 - mean_absolute_percentage_error: 31.7850
Epoch 3/1800
61/61 - 0s - loss: 178539085824.0000 - mean_absolute_percentage_error: 28.3553
Epoch 4/1800
61/61 - 0s - loss: 176724918272.0000 - mean_absolute_percentage_error: 28.4092
Epoch 5/1800
61/61 - 0s - loss: 176561356800.0000 - mean_absolute_percentage_error: 28.3074
Epoch 6/1800
61/61 - 0s - loss: 176142024704.0000 - mean_absolute_percentage_error: 28.2383
Epoch 7/1800
61/61 - 0s - loss: 175937929216.0000 - mean_absolute_percentage_error: 28.2118
Epoch 8/1800
61/61 - 0s - loss: 175836512256.0000 - mean_absolute_percentage_error: 28.1976
Epoch 9/1800
61/61 - 0s - loss: 175770370048.0000 - mean_absolute_percentage_error: 28.1873
Epoch 10/1800
61/61 - 0s - loss: 175720759296.0000 - mean_absolute_percentage_error: 28.1791
Epoch 11/1800
61/61 - 0s - loss: 175680536576.0000 - mean_absolute_percentage_e

Epoch 90/1800
61/61 - 0s - loss: 172553666560.0000 - mean_absolute_percentage_error: 27.6833
Epoch 91/1800
61/61 - 0s - loss: 172510658560.0000 - mean_absolute_percentage_error: 27.6764
Epoch 92/1800
61/61 - 0s - loss: 172463964160.0000 - mean_absolute_percentage_error: 27.6688
Epoch 93/1800
61/61 - 0s - loss: 172421578752.0000 - mean_absolute_percentage_error: 27.6621
Epoch 94/1800
61/61 - 0s - loss: 172379488256.0000 - mean_absolute_percentage_error: 27.6550
Epoch 95/1800
61/61 - 0s - loss: 172339445760.0000 - mean_absolute_percentage_error: 27.6486
Epoch 96/1800
61/61 - 0s - loss: 172302450688.0000 - mean_absolute_percentage_error: 27.6421
Epoch 97/1800
61/61 - 0s - loss: 172256854016.0000 - mean_absolute_percentage_error: 27.6346
Epoch 98/1800
61/61 - 0s - loss: 172215107584.0000 - mean_absolute_percentage_error: 27.6276
Epoch 99/1800
61/61 - 0s - loss: 172173754368.0000 - mean_absolute_percentage_error: 27.6207
Epoch 100/1800
61/61 - 0s - loss: 172131647488.0000 - mean_absolute_pe

Epoch 178/1800
61/61 - 0s - loss: 147399180288.0000 - mean_absolute_percentage_error: 26.3655
Epoch 179/1800
61/61 - 0s - loss: 146063065088.0000 - mean_absolute_percentage_error: 26.2935
Epoch 180/1800
61/61 - 0s - loss: 145761370112.0000 - mean_absolute_percentage_error: 26.1630
Epoch 181/1800
61/61 - 0s - loss: 145566547968.0000 - mean_absolute_percentage_error: 26.0825
Epoch 182/1800
61/61 - 0s - loss: 145492017152.0000 - mean_absolute_percentage_error: 26.1903
Epoch 183/1800
61/61 - 0s - loss: 144150151168.0000 - mean_absolute_percentage_error: 25.9253
Epoch 184/1800
61/61 - 0s - loss: 143628566528.0000 - mean_absolute_percentage_error: 25.9746
Epoch 185/1800
61/61 - 0s - loss: 143283666944.0000 - mean_absolute_percentage_error: 25.6980
Epoch 186/1800
61/61 - 0s - loss: 144370335744.0000 - mean_absolute_percentage_error: 26.1612
Epoch 187/1800
61/61 - 0s - loss: 142252785664.0000 - mean_absolute_percentage_error: 25.7536
Epoch 188/1800
61/61 - 0s - loss: 143030599680.0000 - mean_a

61/61 - 0s - loss: 128045187072.0000 - mean_absolute_percentage_error: 22.7745
Epoch 266/1800
61/61 - 0s - loss: 128715751424.0000 - mean_absolute_percentage_error: 22.7740
Epoch 267/1800
61/61 - 0s - loss: 129529184256.0000 - mean_absolute_percentage_error: 22.9687
Epoch 268/1800
61/61 - 0s - loss: 128064061440.0000 - mean_absolute_percentage_error: 22.9017
Epoch 269/1800
61/61 - 0s - loss: 128939114496.0000 - mean_absolute_percentage_error: 22.8493
Epoch 270/1800
61/61 - 0s - loss: 129144356864.0000 - mean_absolute_percentage_error: 22.8755
Epoch 271/1800
61/61 - 0s - loss: 127227305984.0000 - mean_absolute_percentage_error: 22.4409
Epoch 272/1800
61/61 - 0s - loss: 126855766016.0000 - mean_absolute_percentage_error: 22.5236
Epoch 273/1800
61/61 - 0s - loss: 129066254336.0000 - mean_absolute_percentage_error: 22.8360
Epoch 274/1800
61/61 - 0s - loss: 129532796928.0000 - mean_absolute_percentage_error: 22.9950
Epoch 275/1800
61/61 - 0s - loss: 127041150976.0000 - mean_absolute_percent

Epoch 353/1800
61/61 - 0s - loss: 125279592448.0000 - mean_absolute_percentage_error: 22.5412
Epoch 354/1800
61/61 - 0s - loss: 124224053248.0000 - mean_absolute_percentage_error: 22.2439
Epoch 355/1800
61/61 - 0s - loss: 122833920000.0000 - mean_absolute_percentage_error: 22.0982
Epoch 356/1800
61/61 - 0s - loss: 123811258368.0000 - mean_absolute_percentage_error: 22.1961
Epoch 357/1800
61/61 - 0s - loss: 124111380480.0000 - mean_absolute_percentage_error: 22.0850
Epoch 358/1800
61/61 - 0s - loss: 127687114752.0000 - mean_absolute_percentage_error: 22.8069
Epoch 359/1800
61/61 - 0s - loss: 126437801984.0000 - mean_absolute_percentage_error: 22.6578
Epoch 360/1800
61/61 - 0s - loss: 123947589632.0000 - mean_absolute_percentage_error: 21.9591
Epoch 361/1800
61/61 - 0s - loss: 125613056000.0000 - mean_absolute_percentage_error: 22.4294
Epoch 362/1800
61/61 - 0s - loss: 124856811520.0000 - mean_absolute_percentage_error: 22.2938
Epoch 363/1800
61/61 - 0s - loss: 123168964608.0000 - mean_a

...

Epoch 1400/1800
61/61 - 0s - loss: 117895708672.0000 - mean_absolute_percentage_error: 21.2125
Epoch 1401/1800
61/61 - 0s - loss: 118180225024.0000 - mean_absolute_percentage_error: 21.2871
Epoch 1402/1800
61/61 - 0s - loss: 117805719552.0000 - mean_absolute_percentage_error: 21.2247
Epoch 1403/1800
61/61 - 0s - loss: 117584101376.0000 - mean_absolute_percentage_error: 21.1982
Epoch 1404/1800
61/61 - 0s - loss: 118110330880.0000 - mean_absolute_percentage_error: 21.2705
Epoch 1405/1800
61/61 - 0s - loss: 118054035456.0000 - mean_absolute_percentage_error: 21.2469
Epoch 1406/1800
61/61 - 0s - loss: 118244122624.0000 - mean_absolute_percentage_error: 21.2735
Epoch 1407/1800
61/61 - 0s - loss: 118258761728.0000 - mean_absolute_percentage_error: 21.2723
Epoch 1408/1800
61/61 - 0s - loss: 117991571456.0000 - mean_absolute_percentage_error: 21.2351
Epoch 1409/1800
61/61 - 0s - loss: 117964521472.0000 - mean_absolute_percentage_error: 21.2360
Epoch 1410/1800
61/61 - 0s - loss: 118204309504.00

Epoch 1487/1800
61/61 - 0s - loss: 117947736064.0000 - mean_absolute_percentage_error: 21.4475
Epoch 1488/1800
61/61 - 0s - loss: 117962260480.0000 - mean_absolute_percentage_error: 21.4139
Epoch 1489/1800
61/61 - 0s - loss: 116750467072.0000 - mean_absolute_percentage_error: 21.2401
Epoch 1490/1800
61/61 - 0s - loss: 114473115648.0000 - mean_absolute_percentage_error: 20.7703
Epoch 1491/1800
61/61 - 0s - loss: 116718886912.0000 - mean_absolute_percentage_error: 21.1354
Epoch 1492/1800
61/61 - 0s - loss: 117274787840.0000 - mean_absolute_percentage_error: 21.1114
Epoch 1493/1800
61/61 - 0s - loss: 117644959744.0000 - mean_absolute_percentage_error: 21.1755
Epoch 1494/1800
61/61 - 0s - loss: 116710490112.0000 - mean_absolute_percentage_error: 21.1179
Epoch 1495/1800
61/61 - 0s - loss: 116932706304.0000 - mean_absolute_percentage_error: 21.1102
Epoch 1496/1800
61/61 - 0s - loss: 116699766784.0000 - mean_absolute_percentage_error: 21.0678
Epoch 1497/1800
61/61 - 0s - loss: 116850532352.00

Epoch 1574/1800
61/61 - 0s - loss: 116038541312.0000 - mean_absolute_percentage_error: 21.0052
Epoch 1575/1800
61/61 - 0s - loss: 116320583680.0000 - mean_absolute_percentage_error: 21.0495
Epoch 1576/1800
61/61 - 0s - loss: 117529362432.0000 - mean_absolute_percentage_error: 21.3272
Epoch 1577/1800
61/61 - 0s - loss: 114946981888.0000 - mean_absolute_percentage_error: 20.9203
Epoch 1578/1800
61/61 - 0s - loss: 117481545728.0000 - mean_absolute_percentage_error: 21.3269
Epoch 1579/1800
61/61 - 0s - loss: 115929489408.0000 - mean_absolute_percentage_error: 21.0749
Epoch 1580/1800
61/61 - 0s - loss: 114137047040.0000 - mean_absolute_percentage_error: 20.7569
Epoch 1581/1800
61/61 - 0s - loss: 113894162432.0000 - mean_absolute_percentage_error: 20.6763
Epoch 1582/1800
61/61 - 0s - loss: 116017422336.0000 - mean_absolute_percentage_error: 20.9767
Epoch 1583/1800
61/61 - 0s - loss: 116026925056.0000 - mean_absolute_percentage_error: 20.9989
Epoch 1584/1800
61/61 - 0s - loss: 116283695104.00

Epoch 1661/1800
61/61 - 0s - loss: 112639180800.0000 - mean_absolute_percentage_error: 20.5518
Epoch 1662/1800
61/61 - 0s - loss: 112894509056.0000 - mean_absolute_percentage_error: 20.4957
Epoch 1663/1800
61/61 - 0s - loss: 113694851072.0000 - mean_absolute_percentage_error: 20.5547
Epoch 1664/1800
61/61 - 0s - loss: 115660283904.0000 - mean_absolute_percentage_error: 20.8801
Epoch 1665/1800
61/61 - 0s - loss: 114720620544.0000 - mean_absolute_percentage_error: 20.8074
Epoch 1666/1800
61/61 - 0s - loss: 115426533376.0000 - mean_absolute_percentage_error: 20.9791
Epoch 1667/1800
61/61 - 0s - loss: 112849010688.0000 - mean_absolute_percentage_error: 20.5088
Epoch 1668/1800
61/61 - 0s - loss: 116493975552.0000 - mean_absolute_percentage_error: 21.0988
Epoch 1669/1800
61/61 - 0s - loss: 112825466880.0000 - mean_absolute_percentage_error: 20.5057
Epoch 1670/1800
61/61 - 0s - loss: 116601815040.0000 - mean_absolute_percentage_error: 21.1235
Epoch 1671/1800
61/61 - 0s - loss: 114243887104.00

Epoch 1748/1800
61/61 - 0s - loss: 114791661568.0000 - mean_absolute_percentage_error: 20.8331
Epoch 1749/1800
61/61 - 0s - loss: 114159443968.0000 - mean_absolute_percentage_error: 20.7827
Epoch 1750/1800
61/61 - 0s - loss: 114555445248.0000 - mean_absolute_percentage_error: 20.8425
Epoch 1751/1800
61/61 - 0s - loss: 115828203520.0000 - mean_absolute_percentage_error: 21.0789
Epoch 1752/1800
61/61 - 0s - loss: 112725942272.0000 - mean_absolute_percentage_error: 20.5269
Epoch 1753/1800
61/61 - 0s - loss: 115046318080.0000 - mean_absolute_percentage_error: 20.9656
Epoch 1754/1800
61/61 - 0s - loss: 114678611968.0000 - mean_absolute_percentage_error: 20.8350
Epoch 1755/1800
61/61 - 0s - loss: 114943098880.0000 - mean_absolute_percentage_error: 20.9335
Epoch 1756/1800
61/61 - 0s - loss: 115942776832.0000 - mean_absolute_percentage_error: 21.0575
Epoch 1757/1800
61/61 - 0s - loss: 112624140288.0000 - mean_absolute_percentage_error: 20.5074
Epoch 1758/1800
61/61 - 0s - loss: 114452062208.00